In [87]:
import LiveAuctionProcessor
import importlib
import time
import psycopg2 as pg2
import pandas as pd
from datetime import datetime, timedelta
import math

In [99]:
print ("Connecting to SQL")
conn = pg2.connect(user='postgres',  dbname='penny', host='localhost', port='5432', password='password')

print ("Reading Dataset")
full_df = pd.read_sql ("""Select *  from auction_full where auctionid = 100022191""", conn)
bids_df = pd.read_sql("Select * from bids where auctionid = 100022191 ORDER BY bid", conn)
auctions_df = pd.read_sql("Select * from auctions where qauctionid = 100022191", conn)
bh = [{'bid':b[1]['bid'], 'username':b[1]['username'], 'is_bidomatic': b[1]['is_bidomatic']} for b in bids_df.iterrows()]

Connecting to SQL
Reading Dataset


In [ ]:
prev_df = pd.read_sql("""
with bozo as
(
Select username, max(auctiontime) as max_auctiontime from auction_full group by username
)
select bozo.username,  prev_auction_count0, prev_overbid0, prev_giveup_one0, prev_give_before_six0, 
prev_wins0, prev_bids0, prev_bom_bids0 from bozo left join auction_full on bozo.username=Username0 
and bozo.max_auctiontime=auction_full.auctiontime and auction_full.bids_so_far0=1
"""
, conn)

In [100]:
auction_id = '100022191'
importlib.reload(LiveAuctionProcessor)
auction_dict = {"auction_id":'620712999'}
lap = LiveAuctionProcessor.LiveAuctionProcessor(auction_dict)


In [128]:
#from IPython.display import clear_output

#while (True):
#    clear_output()
#    bh = lap.update()
#    for b in bh[-5:]:
#        print (b)
#    time.sleep(1)

In [102]:
bh

[{'bid': 1, 'username': 'mmmwww', 'is_bidomatic': False},
 {'bid': 2, 'username': 'Wiz1910', 'is_bidomatic': False},
 {'bid': 3, 'username': 'mmmwww', 'is_bidomatic': False},
 {'bid': 4, 'username': 'inmypocket', 'is_bidomatic': False},
 {'bid': 5, 'username': 'jhelmuth', 'is_bidomatic': False},
 {'bid': 6, 'username': 'inmypocket', 'is_bidomatic': False},
 {'bid': 7, 'username': 'mmmwww', 'is_bidomatic': False},
 {'bid': 8, 'username': 'moon0218', 'is_bidomatic': False},
 {'bid': 9, 'username': 'jhelmuth', 'is_bidomatic': False},
 {'bid': 10, 'username': 'BuyItAnyway', 'is_bidomatic': False},
 {'bid': 11, 'username': 'XMAS19722014', 'is_bidomatic': False},
 {'bid': 12, 'username': 'inmypocket', 'is_bidomatic': False},
 {'bid': 13, 'username': 'BuyItAnyway', 'is_bidomatic': False},
 {'bid': 14, 'username': 'inmypocket', 'is_bidomatic': False},
 {'bid': 15, 'username': 'BuyItAnyway', 'is_bidomatic': False},
 {'bid': 16, 'username': 'jhelmuth', 'is_bidomatic': False},
 {'bid': 17, 'usern

In [172]:
bh = bh[:27]

In [236]:
auction_dict = {"auctionid": df.auctionid[0],
  'cashvalue': df.cashvalue[0],
  'cardvalue': df.cardvalue[0],
  'cardtype': df.cardtype[0],
  'bidvalue': df.bidvalue[0],
  'limited_allowed': df.limited_allowed[0],
  'is_locked': False,    #FIXME
  'seconds_left': 531}


In [254]:
my_username =  'DesertVet5702'
auction_dict["idx"]=0
auction_dict["is_winner"] = None
auction_dict["auctiontime"] = datetime.now()+ timedelta(hours=1)
bid = len(bh)+1
auction_dict["bid"] = bid
auction_dict["is_bidomatic"] = True
auction_dict["bids_so_far"] = None
auction_dict["username"] = my_username
auction_dict["fee"]= None
user_dict = {}
cur_opponent = 0
found_users = {}
for r in bh[-1::-1]:
    username = r['username']
    cur_bid = r['bid']
    is_bidomatic = r['is_bidomatic']
    if username == my_username:
        break
    if (username not in found_users.keys()):
        user_dict[cur_opponent]={"username":username, "bids_so_far": 0, "bom_bids_so_far": 0, "distance": bid-cur_bid, "is_bidomatic": is_bidomatic, "bom_streak": 0,  "bom_streak_on": True}
        found_users[username] = cur_opponent
        cur_opponent += 1
    my_row = user_dict[found_users[username]] 
    my_row ["bids_so_far"] += 1
    if (is_bidomatic):
        my_row["bom_bids_so_far"]+=1
        if my_row["bom_streak_on"]:
            my_row["bom_streak"] += 1
    else:
        my_row["bom_streak_on"] = False
for i in range(min(4, len(user_dict))):
    str_i = str(i)
    ud = user_dict[i]
    prev_info = prev_df[prev_df.username==ud["username"]]
    ud["prev_is_new_user"] = False   #FixMe
    ud["prev_auction_count"] = prev_info.prev_auction_count0[prev_info.index[0]]
    ud["prev_overbid"] = prev_info.prev_overbid0[prev_info.index[0]]
    ud["prev_giveup_one"] = prev_info.prev_giveup_one0[prev_info.index[0]]
    ud["prev_give_before_six"] = prev_info.prev_give_before_six0[prev_info.index[0]]
    ud["prev_wins"] = prev_info.prev_wins0[prev_info.index[0]]
    ud["prev_bids"] = prev_info.prev_bids0[prev_info.index[0]]
    ud["prev_bom_bids"] = prev_info.prev_bom_bids0[prev_info.index[0]]
    #, prev_overbid0, prev_giveup_one0, prev_give_before_six0, prev_wins0, prev_bids0, prev_bom_bids0
    ud["perc_to_bin"]=ud["bids_so_far"]/(auction_dict["cashvalue"]*2.5)
    for k in ud:
        auction_dict[k+str_i] = ud[k]
auction_dict["prevusers"] = len(user_dict)
auction_dict["giveup"] = None
auction_dict["eventual_bids"] = None
auction_dict["eventual_win_price"] = None
auction_dict["debut"] = None
auction_dict["bom_streak"] = None
auction_dict["bom_bids_so_far"] = None
auction_dict["perc_to_bin"] = None
auction_dict = {k:[v] for k, v in auction_dict.items()} 


TypeError: can't multiply sequence by non-int of type 'float'

In [245]:

df_out = pd.DataFrame.from_dict(auction_dict)


In [246]:
my_row = df_out.iloc[0,:]


In [252]:
real_row = full_df.loc[full_df.bid == 28,:]

In [253]:
for c in real_row.columns:
    print(c, my_row[c], real_row[c][3])

auctionid 100022191 100022191
is_winner None False
cardtype None None
cashvalue 10 10
cardvalue 0 0
fee None 0.0
bidvalue 25 25
limited_allowed False False
is_locked False False
auctiontime 2020-02-28T23:25:18.067186000 2020-01-16 22:53:19
bid 28 28
is_bidomatic True False
bids_so_far None 1
username DesertVet5702 DesertVet5702
prevusers 7 7
giveup None False
eventual_bids None 5
eventual_win_price None 40
debut None 1
bom_streak None 0
bom_bids_so_far None 0
perc_to_bin None 0.04
username0 BuyItAnyway BuyItAnyway
distance0 1 1.0
is_bidomatic0 False False
bids_so_far0 7 7.0
bom_bids_so_far0 0 0.0
bom_streak0 0 0.0
perc_to_bin0 0.28 0.28


KeyError: 'prev_is_new_user0'

In [73]:
df.columns

Index(['auctionid', 'is_winner', 'cardtype', 'cashvalue', 'cardvalue', 'fee',
       'bidvalue', 'limited_allowed', 'is_locked', 'auctiontime', 'bid',
       'is_bidomatic', 'bids_so_far', 'username', 'prevusers', 'giveup',
       'eventual_bids', 'eventual_win_price', 'debut', 'bom_streak',
       'bom_bids_so_far', 'perc_to_bin', 'username0', 'distance0',
       'is_bidomatic0', 'bids_so_far0', 'bom_bids_so_far0', 'bom_streak0',
       'perc_to_bin0', 'prev_is_new_user0', 'prev_auction_count0',
       'prev_overbid0', 'prev_giveup_one0', 'prev_give_before_six0',
       'prev_wins0', 'prev_bids0', 'prev_bom_bids0', 'username1', 'distance1',
       'is_bidomatic1', 'bids_so_far1', 'bom_bids_so_far1', 'bom_streak1',
       'perc_to_bin1', 'prev_is_new_user1', 'prev_auction_count1',
       'prev_overbid1', 'prev_giveup_one1', 'prev_give_before_six1',
       'prev_wins1', 'prev_bids1', 'prev_bom_bids1', 'username2', 'distance2',
       'is_bidomatic2', 'bids_so_far2', 'bom_bids_so_far2', 

,username,prev_auction_count0,prev_overbid0,prev_giveup_one0,prev_give_before_six0,prev_wins0,prev_bids0,prev_bom_bids0
0,001167,2.0,0.000000,0.500000,0.500000,0.0,69.000000,67.000000
1,007dealbreaker,1.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
2,03fatboy,37.0,0.000000,0.108108,0.972973,0.0,3.810811,0.000000
3,0415Naruto,3.0,0.333333,0.000000,0.000000,0.0,47.666667,19.666667
4,0621bev,7.0,0.000000,0.142857,0.285714,0.0,36.000000,4.571429
...,...,...,...,...,...,...,...,...
9328,zstryjewski,2.0,0.000000,0.000000,0.500000,0.0,11.500000,0.000000
9329,zuber1992,1.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
9330,zwetzig,12.0,0.000000,0.250000,0.666667,0.0,7.333333,0.000000
9331,zz70yy,1.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000


In [170]:
bh

[{'bid': 1, 'username': 'mmmwww', 'is_bidomatic': False},
 {'bid': 2, 'username': 'Wiz1910', 'is_bidomatic': False},
 {'bid': 3, 'username': 'mmmwww', 'is_bidomatic': False},
 {'bid': 4, 'username': 'inmypocket', 'is_bidomatic': False},
 {'bid': 5, 'username': 'jhelmuth', 'is_bidomatic': False},
 {'bid': 6, 'username': 'inmypocket', 'is_bidomatic': False},
 {'bid': 7, 'username': 'mmmwww', 'is_bidomatic': False},
 {'bid': 8, 'username': 'moon0218', 'is_bidomatic': False},
 {'bid': 9, 'username': 'jhelmuth', 'is_bidomatic': False},
 {'bid': 10, 'username': 'BuyItAnyway', 'is_bidomatic': False},
 {'bid': 11, 'username': 'XMAS19722014', 'is_bidomatic': False},
 {'bid': 12, 'username': 'inmypocket', 'is_bidomatic': False},
 {'bid': 13, 'username': 'BuyItAnyway', 'is_bidomatic': False},
 {'bid': 14, 'username': 'inmypocket', 'is_bidomatic': False},
 {'bid': 15, 'username': 'BuyItAnyway', 'is_bidomatic': False},
 {'bid': 16, 'username': 'jhelmuth', 'is_bidomatic': False},
 {'bid': 17, 'usern

6993

222.0